# 📌 300-Level: Response Formatting & Deployment

This notebook focuses on enhancing CrewAI's agent responses by formatting them for UI visualization. Additionally, we will cover containerization and deployment options for running the FastAPI service in a production environment.

## 🎯 Learning Objectives

✔️ Format AI-generated responses for **structured UI representation**.
✔️ Implement a **FastAPI endpoint** to return formatted data.
✔️ **Containerize the application** using Podman/Docker for deployment.

## 🛠️ Installing Dependencies

In [ ]:
!pip install fastapi uvicorn pydantic requests yaml crewai openai

## 🏗️ FastAPI Response Formatting

We'll create an API that formats responses from CrewAI agents into **JSON** or **Markdown** for improved visualization.

In [ ]:
from fastapi import FastAPI, Query
import json
import requests
import logging
import yaml
from crewai import Agent

app = FastAPI()

# ✅ Load Configuration
try:
    with open("configs/config.yaml", "r") as file:
        config = yaml.safe_load(file) or {}
except FileNotFoundError as e:
    logging.error(f"❌ Config file not found: {e}")
    config = {}

FORMATTER_URL = config.get("formatter", {}).get("api_url", "http://localhost:8001/process")
LLM_API_KEY = config.get("llm", {}).get("api_key", None)

def format_response(data, format_type="json"):
    """Formats the response as JSON or Markdown."""
    if format_type == "json":
        return {"formatted": data}
    elif format_type == "markdown":
        return f"```
{data}
```"
    else:
        return {"error": "Unsupported format"}

@app.get("/formatted-response")
def get_formatted_response(format_type: str = Query("json", description="Format type: json or markdown")):
    """API to return a formatted response."""
    sample_response = {"message": "Hello from CrewAI!"}
    return format_response(sample_response, format_type)

@app.get("/")
def home():
    return {"message": "Response Formatting API is running!"}

## 🚀 Running FastAPI Server Locally

In [ ]:
!uvicorn src.api:app --host 0.0.0.0 --port 8000 --reload

## 📦 Containerizing the Application

### 1️⃣ Create a `Containerfile` (or `Dockerfile`)
```dockerfile
FROM python:3.11
WORKDIR /app
COPY . .
RUN pip install --no-cache-dir -r requirements.txt
CMD ["uvicorn", "src.api:app", "--host", "0.0.0.0", "--port", "8000"]
```
### 2️⃣ Build the container
```sh
podman build -t response-formatter:latest .
```
### 3️⃣ Run the container
```sh
podman run -p 8000:8000 response-formatter:latest
```

## 🎯 Summary

✔️ Implemented a **FastAPI service** to format agent responses.
✔️ Supported **JSON and Markdown formatting**.
✔️ Containerized the application for **deployment**.

### **Next Steps:**
We will deploy this service to OpenShift using a Kubernetes pod. 🚀